### 1. Import dependencies

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [3]:
import torchfunc
import flowDatasetV2
from torch.utils.data.sampler import SubsetRandomSampler


In [4]:
torch.__version__
torchfunc.cuda.reset()
torch.cuda.empty_cache()

### 2. Create PyTorch data generators

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        #transforms.Resize((500,500)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        #transforms.Resize((500,500)),
        transforms.ToTensor(),
        normalize
    ]),
}


In [6]:
train_dataset = flowDatasetV2.flowDataset(False,0.2,data_transforms['train'])
test_dataset = flowDatasetV2.flowDataset(True,0.2,data_transforms['validation'])

In [7]:
batch_size = 50
shuffle_dataset = True


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle =True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,  shuffle=True)

In [8]:
dataloaders = {
    'train': train_loader,
    'validation': test_loader
}

### 3. Create the network

In [9]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
device = torch.device("cuda:0")

In [10]:
model = models.resnet50(pretrained=False).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 1024),
               nn.ReLU(inplace=True),
               nn.Linear(1024, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 4)).to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

### 4. Train the model

In [12]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = running_corrects.float() / len(dataloaders[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase, epoch_loss.item(), epoch_acc.item()))
    return model

In [13]:
model_trained = train_model(model, criterion, optimizer, num_epochs=300)

Epoch 1/300
----------


C:\Users\reese\miniconda3\envs\final-year-project\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train loss: 66.8981, acc: 18.2963
validation loss: 66.6473, acc: 14.2857
Epoch 2/300
----------
train loss: 61.9387, acc: 22.2222
validation loss: 53.8504, acc: 26.8571
Epoch 3/300
----------
train loss: 59.4247, acc: 24.1111
validation loss: 51.5786, acc: 26.1429
Epoch 4/300
----------
train loss: 58.3472, acc: 24.6296
validation loss: 48.8035, acc: 26.4286
Epoch 5/300
----------


KeyboardInterrupt: 

### 5. Save and load the model

In [ ]:
torch.save(model_trained.state_dict(), 'models/pytorch/weights.h5')

In [ ]:
model = models.resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 1024),
               nn.ReLU(inplace=True),
               nn.Linear(1024, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 4)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))

### 6. Make predictions on sample test images

In [ ]:
validation_img_paths = ["D:/Flow Videos/dataset/frame_288.jpg",
                        "D:/Flow Videos/dataset/frame_536.jpg",
                        "D:/Flow Videos/dataset/frame_1200.jpg"]
img_list = [Image.open(img_path) for img_path in validation_img_paths]

In [ ]:
validation_batch = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_list])

In [ ]:
pred_logits_tensor = model(validation_batch)
pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()

In [ ]:
fig, axs = plt.subplots(1, len(img_list), figsize=(20, 5))
for i, img in enumerate(img_list):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}% bu, {:.0f}% sl, {:.0f}% ch,{:.0f}% an".format(
    100*pred_probs[i,0], 100*pred_probs[i,1],100*pred_probs[i,2],100*pred_probs[i,3]))
    ax.imshow(img)